In [ ]:
%use mysql(8.0.15)

In [ ]:
import java.sql.Connection
import java.sql.DriverManager
import java.sql.ResultSet

In [ ]:
data class MysqlServer(var host: String, var port: Int,
                       var userName: String, var password: String,
                       var dbName: String) {
    var tableName: String? = null
}

val drive = "com.mysql.cj.jdbc.Driver"

fun connection(mysqlServer: MysqlServer): Connection {
    Class.forName(drive)
    val host = mysqlServer.host
    val port = mysqlServer.port
    val dbName = mysqlServer.dbName
    val url = "jdbc:mysql://$host:$port/$dbName?serverTimezone=UTC&useUnicode=true&characterEncoding=utf-8&useSSL=true"
    return DriverManager.getConnection(url, mysqlServer.userName, mysqlServer.password)
}

fun mysql(mysqlServer: MysqlServer) {
    val tableName = mysqlServer.tableName
    val conn = connection(mysqlServer)
    val tableNames = mutableListOf<String>()
    if (tableName.isNullOrEmpty()) {
        val stmt = conn.createStatement()
        val rs = stmt.executeQuery("show tables;")
        while (rs.next()) {
            tableNames.add(rs.getString(1))
        }
    } else {
        tableNames.add(tableName)
    }

    val dbName = mysqlServer.dbName
    for (name in tableNames) {
        val sql = """
            select * from information_schema.columns
            where table_schema = '$dbName' and table_name = '$name'
            order by ordinal_position
            ;
        """.trimIndent()
        val stmt = conn.createStatement()
        val rs = stmt.executeQuery(sql)
        toMd(rs)
    }
}

fun toMd(rs: ResultSet): String {
    println("|No\t|列名\t|类型\t|空\t|默认值\t|键\t|其他\t|注释|")
    println("|:---\t|:---\t|:---\t|:---\t|:---\t|:---\t|:---\t|:---|")
    while (rs.next()) {
        val int = rs.getInt("ORDINAL_POSITION")
        val string0 = rs.getString("COLUMN_NAME")
        val string1 = rs.getString("COLUMN_TYPE")
        val string2 = rs.getString("IS_NULLABLE")
        val string3 = rs.getString("COLUMN_DEFAULT")
        val string4 = rs.getString("COLUMN_KEY")
        val string5 = rs.getString("EXTRA")
        val string6 = rs.getString("COLUMN_COMMENT")
        println("|$int\t|$string0\t|$string1\t|$string2\t|$string3\t|$string4\t|$string5\t|$string6|")
    }
    return ""
}

val mysqlServer = MysqlServer("luvx", 3306, "root", "????", "boot")
mysqlServer.tableName = "user1"
mysql(mysqlServer)